# Geemap 监督分类及其精度评价

In [1]:
import ee
import geemap
ee.Initialize()
geemap.set_proxy(port="7890")

## 1 数据准备

### 1.1 导入影像&矢量数据

In [2]:
louang_namtha = ee.FeatureCollection('projects/ee-evolto/assets/louang_namtha')
# 导入 Sentinel-2 L2A 影像数据集合
s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')           

In [3]:
# 按照日期、云量和区域进行筛选
s2 = s2.filterDate('2022-01-01', '2022-12-31') \
       .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
       .filterBounds(louang_namtha)

### 1.2 影像去云

In [4]:
# 去云函数
def maskS2clouds(image):
    qa = image.select('QA60')
    # Bits 10 and 11是云，我们要把它mask掉
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 1
    # 这两个标志都应该设置为0，表示条件明确
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
        .And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    # 哨兵的像元值是反射率的10000倍，要除以10000
    return image.updateMask(mask).divide(10000)

In [5]:
collection = s2.map(maskS2clouds).select('B[1-8]')

In [6]:
# 将ee.FeatureCollection转换为ee.Geometry
roi = louang_namtha.geometry()

In [7]:
# 使用中位数合并影像集合并按照研究区域进行裁剪
composite = collection.median().clip(roi)

### 1.3 数据可视化

In [8]:
rgbVis = {
  'min': 0.0,
  'max': 0.3,
  'bands': ['B4', 'B3', 'B2'],
}

In [9]:
# 设置显示样式：color代表边界颜色；fillcolor代表填充颜色
styling = {
    'color': 'red',
    'fillColor': '00000000'
}

In [10]:
Map = geemap.Map()
Map.addLayer(composite, rgbVis, '琅南塔省影像')
Map.addLayer(louang_namtha.style(**styling), {}, '琅南塔省边界')
Map.centerObject(louang_namtha, 9)
Map

Map(center=[20.888098280096127, 101.14354215451829], controls=(WidgetControl(options=['position', 'transparent…

## 2 使用分类器训练数据集

### 2.1 配置分类器参数

In [11]:
# 使用这些波段进行预测
bands  =  [ 'B1' ,  'B2' ,  'B3' ,  'B4' ,  'B5' ,  'B6' ,  'B7' , 'B8']

# 数据集中存储土地覆盖标签的属性
label = 'class_id'

# 导入本地shp格式的土地覆盖分类样本
louang_namtha_sample = geemap.shp_to_ee("E:/researchData/laos/sample/louang_namtha_sample.shp")

跑不通，需要把样本shp的路径改成自己的。

### 2.2 拆分训练集和验证集

In [12]:
# 在图像上叠加点以获得训练, 按照7:3的比例采样训练集和测试集
sample = composite.sampleRegions(**{
    'collection': louang_namtha_sample,
    'properties': [label],
    'scale': 30
})

# 添加一列确定性伪随机数。
sample = sample.randomColumn()

split = 0.7

training = sample.filter(ee.Filter.lt('random', split))
validation = sample.filter(ee.Filter.gte('random', split))

In [13]:
print('训练数据的数量：', training.size().getInfo())
print('验证数据的数量：', validation.size().getInfo())

训练数据的数量： 96364
验证数据的数量： 41254


### 2.3 训练分类器

In [14]:
# 用默认参数训练Cart分类器。
# trainedClassifier = ee.Classifier.smileCart(10).train(training, label, bands)

# 用默认参数训练随机森林分类器。
trainedClassifier = ee.Classifier.smileRandomForest(50).train(training, label, bands)

# 用默认参数训练朴素贝叶斯分类器。
# trainedClassifier = ee.Classifier.smileNaiveBayes().train(training, label, bands)

### 2.4 对图像进行分类

In [15]:
# 用训练时使用的相同波段对图像进行分类。
result = composite.classify(trainedClassifier)

### 2.5 渲染分类图像

In [16]:
# 设置土地分类地物颜色映射
classVis = {
    "min":0,
    "max":5,
    "palette": ["#AB6C28", "#808080", "#A52A2A", "#228B22", "#000000", "#309CD6"]
}

# 定义图例字典
legend_dict = {
    "1 agricultural": "#AB6C28",
    "2 building": "#808080",
    "3 human forest": "#A52A2A",
    "4 nature forest": "#228B22",
    "5 road": "#000000",
    "6 water": "#309CD6",
}

### 2.6 可视化结果

In [17]:
Map.addLayer(result, classVis, '老挝琅南塔省土地分类')
Map.add_legend(legend_title="土地分类", legend_dict=legend_dict)
Map

Map(bottom=58057.0, center=[20.888098280096127, 101.14354215451829], controls=(WidgetControl(options=['positio…

In [18]:
# 获得关于训练分类器的信息。
# print('训练分类器结果', trainedClassifier.explain())

## 3 精度评估

### 3.1 训练数据集

`confusionMatrix()`：根据训练数据集计算分类器的二维混淆矩阵(即:重新替换误差)。矩阵的轴0对应输入类别(即参考数据)，轴1对应输出类别(即分类数据)。行和列从类别0开始，依次增加到最大类别值

In [19]:
# 得到训练样本的混淆矩阵和总体精度。
train_accuracy = trainedClassifier.confusionMatrix()
print('训练误差矩阵:\n')
train_accuracy.getInfo()

训练误差矩阵:



[[619, 0, 0, 5, 0, 0],
 [3, 333, 0, 1, 0, 0],
 [0, 0, 842, 62, 0, 0],
 [1, 0, 1, 94187, 0, 0],
 [2, 2, 0, 3, 15, 0],
 [1, 0, 0, 16, 0, 271]]

In [20]:
print('训练整体准确率:', train_accuracy.accuracy().getInfo())
print('训练整体kappa准确率:', train_accuracy.kappa().getInfo())

训练整体准确率: 0.9989934000249056
训练整体kappa准确率: 0.976926080586554


In [21]:
print('训练整体producers准确率:\n')
train_accuracy.producersAccuracy().getInfo()

训练整体producers准确率:



[[0.9919871794871795],
 [0.9881305637982196],
 [0.9314159292035398],
 [0.999978766097952],
 [0.6818181818181818],
 [0.9409722222222222]]

In [22]:
print('训练整体consumers准确率:\n')
train_accuracy.consumersAccuracy().getInfo()

训练整体consumers准确率:



[[0.9888178913738019,
  0.9940298507462687,
  0.9988137603795967,
  0.9990771580711543,
  1,
  1]]

### 3.2 验证数据集

In [23]:
# 使用训练好的分类器对验证数据集进行分类
validated = validation.classify(trainedClassifier)
validated.first().getInfo()

{'type': 'Feature',
 'geometry': None,
 'id': '0_0',
 'properties': {'B1': 0.04740000143647194,
  'B2': 0.03914999961853027,
  'B3': 0.06404999643564224,
  'B4': 0.05034999921917915,
  'B5': 0.10655000060796738,
  'B6': 0.23804999887943268,
  'B7': 0.28224998712539673,
  'B8': 0.29350000619888306,
  'class_id': 0,
  'classification': 0,
  'random': 0.8785073556842059}}

`errorMatrix`：通过比较集合中的两列(一列是实际值，一列是预测值)，计算出集合的二维误差矩阵。

In [24]:
# 得到验证样本的混淆矩阵和总体精度。
test_accuracy = validated.errorMatrix(label, 'classification')
print('验证误差矩阵:\n')
test_accuracy.getInfo()

验证误差矩阵:



[[258, 6, 0, 30, 0, 0],
 [19, 83, 0, 4, 0, 2],
 [0, 0, 261, 141, 0, 0],
 [10, 1, 27, 40279, 0, 0],
 [0, 3, 0, 5, 0, 0],
 [4, 4, 0, 28, 0, 89]]

In [25]:
print('验证准确性:', test_accuracy.accuracy().getInfo())
print('验证整体kappa准确率:', test_accuracy.kappa().getInfo())

验证准确性: 0.99311581907209
验证整体kappa准确率: 0.8310699526649552


In [26]:
print('验证整体producers准确率:\n')
test_accuracy.producersAccuracy().getInfo()

验证整体producers准确率:



[[0.8775510204081632],
 [0.7685185185185185],
 [0.6492537313432836],
 [0.9990574695537863],
 [0],
 [0.712]]

In [27]:
print('验证整体consumers准确率:\n')
test_accuracy.consumersAccuracy().getInfo()

验证整体consumers准确率:



[[0.8865979381443299,
  0.8556701030927835,
  0.90625,
  0.9948625484723491,
  0,
  0.978021978021978]]

## 4 结果下载

In [33]:
geemap.download_ee_image(
    result,
    f"E:/researchData/laos/louang_namtha_lulc.tif",
    region=louang_namtha.geometry(),
    crs="EPSG:4326",
    scale=100
)

louang_namtha_lulc.tif: |          | 0.00/7.96M (raw) [  0.0%] in 00:00 (eta:     ?)

There is no STAC entry for: None
